*Основные цели этого задания:*
*   Научиться генерировать негативы.
*   Научиться настраивать алгоритмы коллаборативной фильтрации.

Задача: Научиться рекомендовать пользователям фильмы на основе факта просмотра фильмов пользователями. 

In [2]:
import pandas as pd
import numpy as np
import scipy.sparse as sp

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.sparse.linalg import svds
from sklearn.dummy import DummyClassifier
from sklearn.metrics import roc_auc_score

pd.options.display.max_colwidth = 2000
pd.options.display.float_format = '{:.2f}'.format

*1. Для решения задачи будем использовать те же данные, которые были использованы в скринкастах:*
*   MovieLens — источник данных.
*   Предобработанные для обучения данные: история проставления оценок фильмам — ratings_df_sample_2.csv.

In [3]:
ratings = pd.read_csv('2.7ratings_df_sample_2.csv')
movies = pd.read_csv('2.7movies.csv')

In [4]:
ratings=(
    ratings
    .merge(movies, on='movieId')
)

In [5]:
ratings.shape

(6040099, 6)

In [6]:
ratings[:5]

,userId,movieId,rating,timestamp,title,genres
0,54,2,3.00,974918176,Jumanji (1995),Adventure|Children|Fantasy
1,91,2,3.50,1112061358,Jumanji (1995),Adventure|Children|Fantasy
2,116,2,2.00,1132728068,Jumanji (1995),Adventure|Children|Fantasy
3,124,2,2.00,1134476330,Jumanji (1995),Adventure|Children|Fantasy
4,129,2,3.00,1283448701,Jumanji (1995),Adventure|Children|Fantasy


In [7]:
n_users = ratings['userId'].nunique()
n_movies = ratings['movieId'].nunique()
(n_users, n_movies)

(20000, 1000)

In [8]:
%%time
movies_values = ratings['movieId'].unique()

ratings['movieId'] = ratings['movieId'].apply(lambda f: np.where(movies_values == f)[0][0])

CPU times: total: 42.2 s
Wall time: 42.4 s


In [8]:
%%time
users_values = ratings['userId'].unique()

ratings['userId'] = ratings['userId'].apply(lambda f: np.where(users_values == f)[0][0])

CPU times: total: 2min 50s
Wall time: 2min 55s


In [9]:
ratings[:5]

,userId,movieId,rating,timestamp,title,genres
0,0,0,3.00,974918176,Jumanji (1995),Adventure|Children|Fantasy
1,1,0,3.50,1112061358,Jumanji (1995),Adventure|Children|Fantasy
2,2,0,2.00,1132728068,Jumanji (1995),Adventure|Children|Fantasy
3,3,0,2.00,1134476330,Jumanji (1995),Adventure|Children|Fantasy
4,4,0,3.00,1283448701,Jumanji (1995),Adventure|Children|Fantasy


**Вывод по блоку 1: Мы повторили все действия из лекции.**

*2. Предположим, постановка рейтинга — обязательное по итогам просмотра фильмов действие. Основываясь на этом, сгенерируйте новый целевой признак «факт просмотра фильма пользователем», который будет равен 1 для всех пар пользователь * фильм из подгруженного датасета.*

In [10]:
%%time
ratings['viewed'] = ratings['rating'].apply(lambda x: 0 if x==0 else 1)

CPU times: total: 3.36 s
Wall time: 3.45 s


In [11]:
ratings[:5]

,userId,movieId,rating,timestamp,title,genres,viewed
0,0,0,3.00,974918176,Jumanji (1995),Adventure|Children|Fantasy,1
1,1,0,3.50,1112061358,Jumanji (1995),Adventure|Children|Fantasy,1
2,2,0,2.00,1132728068,Jumanji (1995),Adventure|Children|Fantasy,1
3,3,0,2.00,1134476330,Jumanji (1995),Adventure|Children|Fantasy,1
4,4,0,3.00,1283448701,Jumanji (1995),Adventure|Children|Fantasy,1


In [12]:
ratings.describe()

,userId,movieId,rating,timestamp,viewed
count,6040099.00,6040099.00,6040099.00,6040099.00,6040099.00
mean,9115.02,422.90,3.55,1115774334.98,1.00
std,5563.83,283.85,1.00,135843321.30,0.00
min,0.00,0.00,0.50,824835410.00,1.00
25%,4365.00,165.00,3.00,995660158.00,1.00
50%,8679.00,415.00,4.00,1111706240.00,1.00
75%,13758.00,646.00,4.00,1213151458.50,1.00
max,19999.00,999.00,5.00,1427780469.00,1.00


**Вывод по блоку 2: Теперь везде, где рейтинг больше 0 будет стоять 1, как и сказано в задании. Но если посмотреть, то везде в датафрейме стоят 1, 0 проставлено не было.**

*3. А откуда взять «нолики»? В наших данных есть только пары пользователь * фильм, в которых пользователь точно смотрел фильм. Но для обучения модели нужны так называемые «негативы», то есть, пары, где пользователь фильм не смотрел. На практике приходится сталкиваться с необходимостью генерировать их вручную, давайте потренируемся это делать.*
*   Сначала найдите уникальные id всех пользователей и уникальные id всех фильмов.
*   С помощью функции random.choice (документация) сгенерируйте случайные пары пользователь * фильм
*   Поскольку среди сгенерированных пар могут быть и такие, что пользователь в них уже смотрел фильм, сгенерируйте побольше пар, например, удвоенное количество строк из источника. Это может занять пару минут.
*   Среди сгенерированных пар могут быть и дубликаты, удалите их.
*   Оставьте среди сгенерированных пар только те, в которых пользователь фильм не смотрел.
*   Возможно, пар получилось больше, чем нужно, выберите из них столько, сколько у нас строк в исходных данных.
*   Добавьте очищенные сгенерированные пары к исходным данным. Значение целевого признака в них будет равно нулю. Убедитесь, что у вас не появились дубликаты в датасете.

In [13]:
n_users = ratings['userId'].nunique()
n_movies = ratings['movieId'].nunique()
(n_users, n_movies)

(20000, 1000)

**Всего уникальных пользователей 20000, уникальных фильмов 1000, так как мы их до этого предобработали, мы знаем что все значения id от 0 по порядку до 19999.00 (userId) и от 0 до 999.00 (movieId).**

In [14]:
r_len = len(ratings) * 2
ratings_random_user = np.random.choice(ratings['userId'], r_len)
ratings_random_movie = np.random.choice(ratings['movieId'], r_len)
ratings_random_rating = 0

ratings_random = pd.DataFrame({'userId': ratings_random_user, 'movieId': ratings_random_movie, 'rating': ratings_random_rating})
ratings_random=(ratings_random.merge(movies, on='movieId'))

ratings_random

,userId,movieId,rating,title,genres
0,14212,13,0,Balto (1995),Adventure|Animation|Children
1,10004,13,0,Balto (1995),Adventure|Animation|Children
2,2078,13,0,Balto (1995),Adventure|Animation|Children
3,9688,13,0,Balto (1995),Adventure|Animation|Children
4,11899,13,0,Balto (1995),Adventure|Animation|Children
...,...,...,...,...,...
11872762,1885,902,0,Breakfast at Tiffany's (1961),Drama|Romance
11872763,7875,902,0,Breakfast at Tiffany's (1961),Drama|Romance
11872764,6263,902,0,Breakfast at Tiffany's (1961),Drama|Romance
11872765,139,902,0,Breakfast at Tiffany's (1961),Drama|Romance


In [15]:
df = ratings.copy().drop(['timestamp', 'viewed'], axis='columns')
df

,userId,movieId,rating,title,genres
0,0,0,3.00,Jumanji (1995),Adventure|Children|Fantasy
1,1,0,3.50,Jumanji (1995),Adventure|Children|Fantasy
2,2,0,2.00,Jumanji (1995),Adventure|Children|Fantasy
3,3,0,2.00,Jumanji (1995),Adventure|Children|Fantasy
4,4,0,3.00,Jumanji (1995),Adventure|Children|Fantasy
...,...,...,...,...,...
6040094,19955,999,4.00,"Room with a View, A (1986)",Drama|Romance
6040095,19950,999,4.50,"Room with a View, A (1986)",Drama|Romance
6040096,19959,999,3.00,"Room with a View, A (1986)",Drama|Romance
6040097,19967,999,4.50,"Room with a View, A (1986)",Drama|Romance


In [16]:
frames = [df, ratings_random]
df_merged = pd.concat(frames)
df_merged

,userId,movieId,rating,title,genres
0,0,0,3.00,Jumanji (1995),Adventure|Children|Fantasy
1,1,0,3.50,Jumanji (1995),Adventure|Children|Fantasy
2,2,0,2.00,Jumanji (1995),Adventure|Children|Fantasy
3,3,0,2.00,Jumanji (1995),Adventure|Children|Fantasy
4,4,0,3.00,Jumanji (1995),Adventure|Children|Fantasy
...,...,...,...,...,...
11872762,1885,902,0.00,Breakfast at Tiffany's (1961),Drama|Romance
11872763,7875,902,0.00,Breakfast at Tiffany's (1961),Drama|Romance
11872764,6263,902,0.00,Breakfast at Tiffany's (1961),Drama|Romance
11872765,139,902,0.00,Breakfast at Tiffany's (1961),Drama|Romance


In [17]:
len(df_merged) - len(df_merged.drop_duplicates(subset=['userId', 'movieId'], keep='first'))

6927057

**Всего найдено дубликатов по 2м столбцам (userId и movieId), мы их удалим по условию сохранить первое вхождение, так как вначале у нас исходный датафрейм.**

In [18]:
df_merged = df_merged.drop_duplicates(subset=['userId', 'movieId'], keep='first')

In [19]:
df_merged

,userId,movieId,rating,title,genres
0,0,0,3.00,Jumanji (1995),Adventure|Children|Fantasy
1,1,0,3.50,Jumanji (1995),Adventure|Children|Fantasy
2,2,0,2.00,Jumanji (1995),Adventure|Children|Fantasy
3,3,0,2.00,Jumanji (1995),Adventure|Children|Fantasy
4,4,0,3.00,Jumanji (1995),Adventure|Children|Fantasy
...,...,...,...,...,...
11872761,12372,902,0.00,Breakfast at Tiffany's (1961),Drama|Romance
11872762,1885,902,0.00,Breakfast at Tiffany's (1961),Drama|Romance
11872763,7875,902,0.00,Breakfast at Tiffany's (1961),Drama|Romance
11872764,6263,902,0.00,Breakfast at Tiffany's (1961),Drama|Romance


**Осталось 10985809 строк, добавим наш столбец viewed обратно**

In [20]:
df_merged['viewed'] = df_merged['rating'].apply(lambda x: 0 if x==0 else 1)

C:\Users\1will\AppData\Local\Temp\ipykernel_2704\1407039748.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged['viewed'] = df_merged['rating'].apply(lambda x: 0 if x==0 else 1)


In [21]:
df_merged.describe()

,userId,movieId,rating,viewed
count,10985809.00,10985809.00,10985809.00,10985809.00
mean,9460.21,453.86,1.95,0.55
std,5651.47,285.54,1.92,0.50
min,0.00,0.00,0.00,0.00
25%,4600.00,201.00,0.00,0.00
50%,9178.00,448.00,2.00,1.00
75%,14259.00,688.00,4.00,1.00
max,19999.00,999.00,5.00,1.00


**Вывод по блоку 3: Вывод Теперь в viewed есть и 0 и 1**

*4. Подготовьте датасет к обучению: отделите тестовую часть от тренировочной.*

In [34]:
X = df_merged.drop(['viewed'], axis=1)
y = df_merged['viewed']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.10, random_state=42)

**Вывод по блоку 4: Разделили на тестовую и обучающую выборки**

*5. Обучите dummy-model. Пусть она будет возвращать случайную вероятность принадлежности классу 1. Для этого можете использовать функцию random.random (документация). Оцените ее качество какой-то метрикой на свой вкус. Необходимо прогнозировать именно вероятность, чтобы была возможность ранжировать по ней варианты для рекомендации лучшего контента пользователю.*

In [35]:
def dum_model(features):
    return [np.random.random() for item in range(len(features))]

dum_pred_train = dum_model(X_train)
dum_pred_test = dum_model(X_test)

In [36]:
roc_auc_dum_train = roc_auc_score(y_train, dum_pred_train)
roc_auc_dum_test = roc_auc_score(y_test, dum_pred_test)
print(f'ROC-AUC на тренировочной {roc_auc_dum_train}')
print(f'ROC-AUC на тестовой {roc_auc_dum_test}')

ROC-AUC на тренировочной 0.5001718313562331
ROC-AUC на тестовой 0.5007658416756999


**Вывод по блоку 5: Обучили dummy-model, которая возвращает случайные вероятности принадлежности к классу, по метрике ROC-AUC мы видим показатель 0.5, т.е. в половине случаев dummy-model угадала принадлежность к классу.**

*6. Реализуйте три алгоритма коллаборативной фильтрации: user-, item-based и алгоритм на основе матричной факторизации. Оцените их качество и адекватность. Если качество недостаточно хорошее, попробуйте варьировать параметры: количество похожих пользователей/фильмов, количество элементов в матрицах при матричном разложении.*

In [37]:
train_data, test_data = train_test_split(df_merged, test_size=0.01)

print('Train shape: {}'.format(train_data.shape))
print('Test shape: {}'.format(test_data.shape))

Train shape: (10875950, 6)
Test shape: (109859, 6)


In [26]:
n_users_m = train_data['userId'].nunique()
n_movies_m = train_data['movieId'].nunique()
(n_users_m, n_movies_m)

(20000, 1000)

In [38]:
train_data_matrix = np.array(pd.pivot_table(train_data, values='viewed', index='userId', columns='movieId', fill_value=0))
train_data_matrix.shape

(20000, 1000)

In [39]:
train_data_matrix

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]], dtype=int64)

Расчет попарного косинусного расстояния для пользователей и для фильмов

In [40]:
user_sim = cosine_distances(train_data_matrix)
user_sim

array([[0.        , 0.46966875, 0.44044685, ..., 0.96679351, 0.97424489,
        0.99591015],
       [0.46966875, 0.        , 0.38979745, ..., 0.8125483 , 0.88692013,
        0.77682236],
       [0.44044685, 0.38979745, 0.        , ..., 0.78613317, 0.89970282,
        0.75742015],
       ...,
       [0.96679351, 0.8125483 , 0.78613317, ..., 0.        , 0.89927213,
        0.62570874],
       [0.97424489, 0.88692013, 0.89970282, ..., 0.89927213, 0.        ,
        0.7320289 ],
       [0.99591015, 0.77682236, 0.75742015, ..., 0.62570874, 0.7320289 ,
        0.        ]])

In [41]:
movie_sim = cosine_distances(train_data_matrix.T)
movie_sim

array([[0.        , 0.39262445, 0.38055109, ..., 0.75439597, 0.59110325,
        0.78019071],
       [0.39262445, 0.        , 0.21160057, ..., 0.66486715, 0.53271617,
        0.6816813 ],
       [0.38055109, 0.21160057, 0.        , ..., 0.6738041 , 0.48875393,
        0.72281546],
       ...,
       [0.75439597, 0.66486715, 0.6738041 , ..., 0.        , 0.77950514,
        0.64971218],
       [0.59110325, 0.53271617, 0.48875393, ..., 0.77950514, 0.        ,
        0.8422052 ],
       [0.78019071, 0.6816813 , 0.72281546, ..., 0.64971218, 0.8422052 ,
        0.        ]])

In [42]:
print('user_sim' , user_sim.shape)
print('movie_sim' , movie_sim.shape)

user_sim (20000, 20000)
movie_sim (1000, 1000)


*Алгоритм user-based*

In [43]:
top=10
top_similar_users = []
for i in range(n_users_m):
    neighbors = (user_sim[i]).argsort()[1:top + 1]
    top_similar_users.append(
        train_data_matrix[neighbors]
    )
top_similar_users = np.array(top_similar_users)

In [44]:
print('Количество  пользователей, количество соседей, количество фильмов' , top_similar_users.shape)

Количество  пользователей, количество соседей, количество фильмов (20000, 10, 1000)


In [45]:
predicted_viewed_user_based = top_similar_users.mean(1)
predicted_viewed_user_based.shape

(20000, 1000)

Предсказание для тестового датасета:

In [79]:
def viewed_pred(n):
    return round((n * 2) / 2)

test_data['predict_user_based'] = test_data.apply(
    lambda f: viewed_pred(predicted_viewed_user_based[f['userId'], f['movieId']]), axis = 1
)

print(f"ROC AUC metric: {np.sqrt(roc_auc_score(test_data['predict_user_based'], test_data['viewed']))}")

ROC AUC metric: 0.8642428629308494


**Мы видим, что результат у user-based алгоритма гораздо лучше, чем у dummy-model**

*Алгоритм item-based*

In [57]:
top=10
top_similar_movies = []
for i in range(n_movies):
    neighbors = (movie_sim[i]).argsort()[1:top + 1]
    top_similar_movies.append(
        train_data_matrix.T[neighbors]
    )
    
top_similar_movies = np.array(top_similar_movies)

top_similar_movies.shape

(1000, 10, 20000)

In [58]:
predicted_viewed_item_based = top_similar_movies.mean(1).T
predicted_viewed_item_based.shape

(20000, 1000)

In [80]:
test_data['predict_item_based'] = test_data.apply(
    lambda f: viewed_pred(predicted_viewed_item_based[f['userId'], f['movieId']]), axis = 1
)

print(f"ROC AUC metric: {np.sqrt(roc_auc_score(test_data['predict_item_based'], test_data['viewed']))}")

ROC AUC metric: 0.8568728708650855


**item-based алгоритм отработал на уровне (немного ниже, но не критично ниже) user-based алгоритма**

*Алгоритм на основе матричного разложения*

In [60]:
train_data_matrix.shape

(20000, 1000)

In [61]:
train_data_matrix = train_data_matrix.astype(float)

In [102]:
u, s, vh = svds(train_data_matrix, k=80)
s_diag_matrix = np.diag(s)

users = np.dot(u, s_diag_matrix)
items = vh.T

In [103]:
print('diag(s).shape' , np.diag(s).shape)
print('vh.shape', vh.shape)
print('users.shape' , users.shape)
print('items.shape' , items.shape)

diag(s).shape (80, 80)
vh.shape (80, 1000)
users.shape (20000, 80)
items.shape (1000, 80)


In [104]:
test_data['svd_predictions'] = test_data.apply(
    lambda f: viewed_pred(np.dot(users[f['userId']], items[f['movieId']])), axis = 1
)

print(f"ROC AUC metric: {np.sqrt(roc_auc_score(test_data['svd_predictions'], test_data['viewed']))}")

ROC AUC metric: 0.8776009669794868


**Алгоритм на основе матричного разложения отработал лучше остальных** 

*7. Опишите вывод, содержащий информацию о том, какой алгоритм проявил себя лучше всего.*

Какая работа была проделана:
*   Сделали короткий анализ датасета, проверили на пропуски, заполнили колонку viewed 1
*   Все id пользователей и фильмов привели в вид от 0 до максимального уникального значения 
*   Добавили фильмы, которые не смотрел пользователь, заполнили колонку viewed 0
*   Посмотрели как работает dummy-model, user-based, item-based и svd, проверили качество метрикой ROC-AUC

Теперь рассмотрим все алгоритмы вместе:
*   dummy-model - 0,5
*   user-based - 0.8642428629308494
*   item-based - 0.8568728708650855
*   svd - 0.8776009669794868

Как мы видим, Алгоритм на основе матричного разложения (svd) имеет наивысший результат среди остальных по метрике ROC-AUC, и как следствие, справился с задачей рекомендации фильмов лучше, потом user-based и item-based алгоритмы, которые в целом тоже справились неплохо, и dummy-model с результатом 0.5.